# Isogeometric analysis with `IGA.jl`
This tutorial shows how to integrate FerriteAssembly with the
isogeometric analysis toolbox IGA.jl, directly based on `IGA.jl`'s
[Infinite plate with hole](https://lijas.github.io/IGA.jl/dev/examples/plate_with_hole/)
example. Hence, please see there for further documentation details and important remarks
regarding IGA.

The example considers solving a plate with a hole. A quarter of a plate is considered via symmetry
boundary conditions, and a tensile load is applied on one edge.
The full script without intermediate comments is available at the
bottom of this page.

Start by loading the necessary packages

In [1]:
using Ferrite, IGA, LinearAlgebra, FerriteAssembly
import FerriteAssembly.ExampleElements: ElasticPlaneStrain

## Setup
To clarify the differences, we split the setup into `IGA.jl`, `Ferrite.jl`,
and `FerriteAssembly.jl` specific parts.
### `IGA.jl` setup
We begin by generating the mesh by using `IGA.jl`'s built-in mesh generators,
specifically a "plate with hole".

In [2]:
function create_mesh(; nels = (20,10), order)
    nurbsmesh = generate_nurbs_patch(:plate_with_hole, nels, order)
    grid = BezierGrid(nurbsmesh)

    addfacetset!(grid, "left", (x) -> x[1] ≈ -4.0)
    addfacetset!(grid, "bot", (x) -> x[2] ≈ 0.0)
    addfacetset!(grid, "right", (x) -> x[1] ≈ 0.0)

    return grid
end
order = 2
grid = create_mesh(; order);

Create the `IGA.jl` cell and facet values for storing the
the `IGA.jl` shape function values and gradients.

In [3]:
ip = IGAInterpolation{RefQuadrilateral, order}()
qr_cell = QuadratureRule{RefQuadrilateral}(4)
qr_face = FacetQuadratureRule{RefQuadrilateral}(3)

cv = BezierCellValues(qr_cell, ip^2);
fv = BezierFacetValues(qr_face, ip^2);

### `Ferrite.jl` setup
Distribute dofs as normal

In [4]:
dh = DofHandler(grid)
add!(dh, :u, ip^2)
close!(dh);

And allocate system matrices and vectors

In [5]:
a = zeros(ndofs(dh))
r = zeros(ndofs(dh))
K = allocate_matrix(dh);

Before adding boundary conditions, starting with Dirichlet:
1) Bottom face should only be able to move in x-direction
2) Right boundary should only be able to move in y-direction

In [6]:
ch = ConstraintHandler(dh);
add!(ch, Dirichlet(:u, getfacetset(grid, "bot"), Returns(0.0), 2))
add!(ch, Dirichlet(:u, getfacetset(grid, "right"), Returns(0.0), 1))
close!(ch)
update!(ch, 0.0);

### `FerriteAssembly.jl` setup
Neumann boundary conditions are added using `FerriteAssembly`'s
`LoadHandler`. We apply outwards traction on the left surface,
and take the negative value since r = fint - fext.

In [7]:
traction = Vec((-10.0, 0.0))
lh = LoadHandler(dh)
add!(lh, Neumann(:u, fv, getfacetset(grid, "left"), Returns(-traction)));

material = ElasticPlaneStrain(;E=100.0, ν=0.3)
domain = DomainSpec(dh, material, cv)
buffer = setup_domainbuffer(domain);

## Assembly and solution
We first assemble the equation system,

In [8]:
assembler = start_assemble(K, r)
apply!(a, ch)
work!(assembler, buffer; a=a)
apply!(r, lh, 0.0);

before solving it,

In [9]:
apply_zero!(K, r, ch)
a .-= K\r
apply!(a, ch);

## Postprocessing
First, the stresses in each integration point are calculated by using the Integrator.

In [10]:
struct CellStress{TT}
    s::Vector{Vector{TT}}
end
function CellStress(grid::Ferrite.AbstractGrid)
    TT = typeof(zero(SymmetricTensor{2, Ferrite.getspatialdim(grid)}))
    return CellStress([TT[] for _ in 1:getncells(grid)])
end

function FerriteAssembly.integrate_cell!(stress::CellStress, state, ae, material, cv, cb)
    σ = stress.s[cellid(cb)]
    for q_point in 1:getnquadpoints(cv)
        ϵ = function_symmetric_gradient(cv, q_point, ae)
        push!(σ, material.C⊡ϵ)
    end
end
cellstresses = CellStress(grid)
integrator = Integrator(cellstresses)
work!(integrator, buffer; a=a);

Now we want to export the results to VTK. So we project the stresses at
the quadrature points to the nodes using the L2Projector from Ferrite.
Currently, however, IGA doesn't support L2 projection.
```julia
# projector = L2Projector(ip, grid)
# σ_nodes = IGA.igaproject(projector, cellstresses.s, qr_cell; project_to_nodes=true);
```

Output results to VTK

In [11]:
IGA.VTKIGAFile("plate_with_hole.vtu", grid) do vtk
    write_solution(vtk, dh, a)
end

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*